Problem 1

In [1]:
import requests
from scrapy.http import TextResponse
import time
import pandas as pd

In [2]:
def author_scraper(response):
    authors=response.css("small::text").extract()
    return authors

In [3]:
def tag_scraper(response):
    return [i.css("a.tag::text").extract() for i in response.css("div.tags")]

In [4]:
def quote_scraper(response):
    quotes=response.css("span[class='text']::text").extract()
    return quotes

In [5]:
def author_page_scraper(response):
    base_url = "http://quotes.toscrape.com"
    author_pages = response.css("small.author ~ a::attr(href)").extract()
    return [base_url + i for i in author_pages]

In [6]:
url = "http://quotes.toscrape.com/"
base_url = "http://quotes.toscrape.com"
all_pages = []
while(True):
    page = requests.get(url)
    response = TextResponse(body=page.text,url = "http://quotes.toscrape.com/",encoding = "utf-8")
    authors = author_scraper(response)
    tags = tag_scraper(response)
    quotes = quote_scraper(response)
    author_pages = author_page_scraper(response)
    all_pages.append(pd.DataFrame({"Authors":authors,"Tags":tags,"Quotes":quotes,"Authors_pages":author_pages}))
    next = response.css("li[class='next'] a::attr(href)").extract()
    if not next:
        break
    else:
        url = base_url + next[0]
        time.sleep(2)
        
pd.concat(all_pages)

,Authors,Tags,Quotes,Authors_pages
0,Albert Einstein,"[change, deep-thoughts, thinking, world]",“The world as we have created it is a process ...,http://quotes.toscrape.com/author/Albert-Einstein
1,J.K. Rowling,"[abilities, choices]","“It is our choices, Harry, that show what we t...",http://quotes.toscrape.com/author/J-K-Rowling
2,Albert Einstein,"[inspirational, life, live, miracle, miracles]",“There are only two ways to live your life. On...,http://quotes.toscrape.com/author/Albert-Einstein
3,Jane Austen,"[aliteracy, books, classic, humor]","“The person, be it gentleman or lady, who has ...",http://quotes.toscrape.com/author/Jane-Austen
4,Marilyn Monroe,"[be-yourself, inspirational]","“Imperfection is beauty, madness is genius and...",http://quotes.toscrape.com/author/Marilyn-Monroe
...,...,...,...,...
5,Harper Lee,[better-life-empathy],“You never really understand a person until yo...,http://quotes.toscrape.com/author/Harper-Lee
6,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...",“You have to write the book that wants to be w...,http://quotes.toscrape.com/author/Madeleine-LE...
7,Mark Twain,[truth],“Never tell the truth to people who are not wo...,http://quotes.toscrape.com/author/Mark-Twain
8,Dr. Seuss,[inspirational],"“A person's a person, no matter how small.”",http://quotes.toscrape.com/author/Dr-Seuss


Problem 2

In [7]:
url_2 = "https://www.imdb.com/chart/moviemeter/"
page_2 = requests.get(url_2)
response_2 = TextResponse(body=page_2.text,url = "https://www.imdb.com/chart/moviemeter/",encoding = "utf-8")

In [8]:
def title_scraper(response):
    titles=response_2.css("td[class='titleColumn'] a::text").extract()
    return titles

In [9]:
def year_scraper(response):
    years=response_2.css("td[class='titleColumn'] a ~ span::text").extract()
    new_years = []
    for i in years:
        new_year = i.replace("(","")
        new_year = new_year.replace(")","")
        new_years.append(new_year)
    return new_years

In [10]:
def ranking_scraper(response):
    rankings=response_2.css("span:not([class='secondaryInfo']) div[class='velocity']::text").extract()
    new_rankings = []
    for i in rankings:
        new_ranking = i.replace("\n","")
        if(new_ranking):
            if(new_ranking.find("(")==-1):
                new_rankings.append(new_ranking)
            else:
                index = new_ranking.index("(")
                new_ranking = new_ranking[0:index]
                new_rankings.append(new_ranking)
    return new_rankings

In [17]:
def rating_scraper(response):
    rows = response_2.css("tbody[class='lister-list'] tr")
    new_ratings=[]
    for i in rows:
        rating=i.css("td[class='ratingColumn imdbRating'] strong::text").extract()
        if(not rating):
            new_ratings.append("")
        else:
            new_ratings.append(rating[0])
    return new_ratings

In [12]:
def rating_scraper(response):
    ratings=response_2.css("td[class='ratingColumn imdbRating'] strong::text").extract()
    return ratings

In [13]:
def link_scraper(response):
    base_url = "https://www.imdb.com/"
    links=response_2.css("td[class='titleColumn'] a::attr(href)").extract()
    return [base_url + i for i in links]

In [20]:
titles = title_scraper(response_2)
years = year_scraper(response_2)
ranking = ranking_scraper(response_2)
ratings = rating_scraper(response_2)
links = link_scraper(response_2)
print((ratings))
pd.DataFrame({"Titles":titles,"Years":years,"Ranking":ranking,"Ratings":ratings,"Links":links})

['3.5', '6.7', '4.1', '7.2', '7.9', '5.3', '7.2', '', '7.5', '8.6', '3.6', '7.7', '6.4', '8.5', '7.9', '7.6', '', '6.1', '', '6.5', '8.4', '6.3', '7.1', '8.3', '6.7', '3.6', '7.1', '6.6', '7.1', '', '5.7', '5.7', '5.7', '9.3', '8.4', '8.1', '8.2', '5.4', '', '6.8', '8.8', '7.3', '8.1', '7.9', '6.8', '7.5', '8.8', '7.3', '6.7', '6.1', '7.9', '6.7', '5.8', '9.2', '6.8', '8.6', '6.4', '8.1', '', '7.6', '6.7', '8.5', '', '8.1', '8.6', '9.0', '4.1', '', '7.3', '8.2', '5.7', '6.1', '', '6.6', '8.9', '', '8.4', '', '6.4', '7.4', '6.9', '6.7', '', '6.6', '6.0', '8.7', '4.8', '7.6', '7.7', '7.0', '', '8.1', '7.8', '5.7', '8.3', '6.2', '5.7', '3.7', '6.8', '7.5']


,Titles,Years,Ranking,Ratings,Links
0,365 dni,2020,1,3.5,https://www.imdb.com//title/tt10886166/
1,Da 5 Bloods,2020,2,6.7,https://www.imdb.com//title/tt9777644/
2,Artemis Fowl,2020,3,4.1,https://www.imdb.com//title/tt3089630/
3,The King of Staten Island,2020,4,7.2,https://www.imdb.com//title/tt9686708/
4,Knives Out,2019,5,7.9,https://www.imdb.com//title/tt8946378/
...,...,...,...,...,...
95,The Rising Hawk,2019,96,6.2,https://www.imdb.com//title/tt7439064/
96,Bloodshot,2020,97,5.7,https://www.imdb.com//title/tt1634106/
97,The Room,2003,98,3.7,https://www.imdb.com//title/tt0368226/
98,Emma.,2020,99,6.8,https://www.imdb.com//title/tt9214832/


Problem 3

In [21]:
def book_title_scraper(response):
    titles=response.css("article[class='product_pod'] h3 a::attr(title)").extract()
    return titles

In [22]:
def book_rating_scraper(response):
    rows = response.css("article[class='product_pod']")
    ratings = []
    for i in rows:
        one_star = i.css("p[class='star-rating One']")
        two_star = i.css("p[class='star-rating Two']")
        three_star = i.css("p[class='star-rating Three']")
        four_star = i.css("p[class='star-rating Four']")
        five_star = i.css("p[class='star-rating Five']")
        if(len(one_star)==1):
            ratings.append("One")
        elif(len(two_star)==1):
            ratings.append("Two")
        elif(len(three_star)==1):
            ratings.append("Three")
        elif(len(four_star)==1):
            ratings.append("Four")
        else:
            ratings.append("Five")
    return ratings

In [23]:
def book_price_scraper(response):
    prices=response.css("p[class='price_color']::text").extract()
    return [i.replace("Â£","") for i in prices]

In [24]:
def book_url_scraper(response):
    base_url = "http://books.toscrape.com/";
    urls=response.css("article[class='product_pod'] h3 a::attr(href)").extract()
    return [base_url + i for i in urls]

In [25]:
def book_image_url_scraper(response):
    base_url = "http://books.toscrape.com/";
    image_urls=response.css("div[class='image_container'] a img::attr(src)").extract()
    return [base_url + i for i in image_urls]

In [26]:
def in_stock_scraper(response):
    in_stocks=response.css("p[class='instock availability']::text").extract()
    new_stocks = []
    for i in in_stocks:
        new_stock = i.replace("\n","")
        new_stock = new_stock.strip()
        if(new_stock):
            new_stocks.append(new_stock)
    return new_stocks

In [27]:
all_pages = []
for i in range(1,51):
    url_3 = f"http://books.toscrape.com/catalogue/page-{i}.html"
    page_3 = requests.get(url_3)
    response_3 = TextResponse(body=page_3.text,url = url_3,encoding = "utf-8")
    book_titles = book_title_scraper(response_3)
    book_ratings = book_rating_scraper(response_3)
    book_prices = book_price_scraper(response_3)
    book_urls = book_url_scraper(response_3)
    book_image_urls = book_image_url_scraper(response_3)
    book_in_stock= in_stock_scraper(response_3)
    time.sleep(1)

    all_pages.append(pd.DataFrame({"Title":book_titles,"Rating":book_ratings,"Price":book_prices,"Book Url":book_urls,"Image Url":book_image_urls,"In Stock":book_in_stock}))
pd.concat(all_pages)


,Title,Rating,Price,Book Url,Image Url,In Stock
0,A Light in the Attic,Three,51.77,http://books.toscrape.com/a-light-in-the-attic...,http://books.toscrape.com/../media/cache/2c/da...,In stock
1,Tipping the Velvet,One,53.74,http://books.toscrape.com/tipping-the-velvet_9...,http://books.toscrape.com/../media/cache/26/0c...,In stock
2,Soumission,One,50.10,http://books.toscrape.com/soumission_998/index...,http://books.toscrape.com/../media/cache/3e/ef...,In stock
3,Sharp Objects,Four,47.82,http://books.toscrape.com/sharp-objects_997/in...,http://books.toscrape.com/../media/cache/32/51...,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,http://books.toscrape.com/sapiens-a-brief-hist...,http://books.toscrape.com/../media/cache/be/a5...,In stock
...,...,...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,One,55.53,http://books.toscrape.com/alice-in-wonderland-...,http://books.toscrape.com/../media/cache/96/ee...,In stock
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,57.06,http://books.toscrape.com/ajin-demi-human-volu...,http://books.toscrape.com/../media/cache/09/7c...,In stock
17,A Spy's Devotion (The Regency Spies of London #1),Five,16.97,http://books.toscrape.com/a-spys-devotion-the-...,http://books.toscrape.com/../media/cache/1b/5f...,In stock
18,1st to Die (Women's Murder Club #1),One,53.98,http://books.toscrape.com/1st-to-die-womens-mu...,http://books.toscrape.com/../media/cache/2b/41...,In stock
